In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from dtuimldmtools import mcnemar

### Loading data and preprocessing

In [9]:
df = pd.read_csv("HR_data.csv")
df

,Unnamed: 0,HR_Mean,HR_Median,HR_std,HR_Min,HR_Max,HR_AUC,Round,Phase,Individual,Puzzler,Frustrated,Cohort
0,0,77.965186,78.000,3.345290,73.23,83.37,22924.945,round_3,phase3,1,1,1,D1_1
1,1,70.981097,70.570,2.517879,67.12,78.22,21930.400,round_3,phase2,1,1,5,D1_1
2,2,73.371959,73.360,3.259569,67.88,80.22,21647.085,round_3,phase1,1,1,0,D1_1
3,3,78.916822,77.880,4.054595,72.32,84.92,25258.905,round_2,phase3,1,1,1,D1_1
4,4,77.322226,74.550,6.047603,70.52,90.15,23890.565,round_2,phase2,1,1,5,D1_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,163,73.594539,72.380,9.474556,57.43,93.53,21482.985,round_4,phase2,14,0,8,D1_2
164,164,57.839897,54.130,6.796647,52.97,74.14,16825.740,round_4,phase1,14,0,0,D1_2
165,165,64.237295,65.195,3.589241,58.97,72.63,18691.065,round_1,phase3,14,0,1,D1_2
166,166,70.834320,70.440,2.391160,66.65,76.07,20753.005,round_1,phase2,14,0,4,D1_2


In [ ]:
df = df.drop(columns=["Cohort", "Unnamed: 0"])

num_cols = ["HR_Mean", "HR_Median","HR_std", "HR_Min", "HR_Max", "HR_AUC"]
cat_cols = ["Round", "Phase"] # Puzzler is already binarized, but is also categorical

df = pd.get_dummies(df, columns=cat_cols, drop_first=True).astype(int) # One out of K encoding

df["y"] = (df["Frustrated"] > 2).astype(int) # 0-2 negative, 3-10 positive

df

,HR_Mean,HR_Median,HR_std,HR_Min,HR_Max,HR_AUC,Individual,Puzzler,Frustrated,Round_round_2,Round_round_3,Round_round_4,Phase_phase2,Phase_phase3,y
0,77,78,3,73,83,22924,1,1,1,0,1,0,0,1,0
1,70,70,2,67,78,21930,1,1,5,0,1,0,1,0,1
2,73,73,3,67,80,21647,1,1,0,0,1,0,0,0,0
3,78,77,4,72,84,25258,1,1,1,1,0,0,0,1,0
4,77,74,6,70,90,23890,1,1,5,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,73,72,9,57,93,21482,14,0,8,0,0,1,1,0,1
164,57,54,6,52,74,16825,14,0,0,0,0,1,0,0,0
165,64,65,3,58,72,18691,14,0,1,0,0,0,0,1,0
166,70,70,2,66,76,20753,14,0,4,0,0,0,1,0,1


In [11]:
X = df.drop(columns=["y", "Frustrated", "Individual"]).values
y = df["y"].values
groups = df["Individual"].values   # Getting the groups for leave one individual out

### Defining models and leave on individual out cross-validation

In [12]:
lr = LogisticRegression(
    penalty="l2",
    C=1.0,
    max_iter=1000
)

rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=6,
    criterion="gini",
    random_state=1351
)

logo = LeaveOneGroupOut() # Leave one individual out cross validaiton

scaler = StandardScaler() # Scaler to standardize only on training data

In [13]:
y_true, yhat_lr, yhat_rf, yhat_base = [], [], [], [] # keeping track of all predictions

fold_acc_lr, fold_acc_rf, fold_acc_base = [], [], [] # Keeping track of accuracies in each fold

for train_idx, test_idx in logo.split(X, y, groups):
    Xtr, Xte = X[train_idx], X[test_idx]
    ytr, yte = y[train_idx], y[test_idx]

    scaler.fit(Xtr) # Scale only on Xtr
    Xtr = scaler.transform(Xtr)
    Xte = scaler.transform(Xte)
    
    # Logistic regression
    lr_fitted = lr.fit(Xtr, ytr)
    pred_lr = lr_fitted.predict(Xte)
    yhat_lr.extend(pred_lr)

    # Random forest
    rf_fitted = rf.fit(Xtr, ytr)
    pred_rf = rf_fitted.predict(Xte)
    yhat_rf.extend(pred_rf)
    
    # Baseline
    pred_base = [0] * len(yte)
    yhat_base.extend(pred_base)
    fold_acc_base.append(accuracy_score(yte, pred_base))    

    y_true.extend(yte)
    
    fold_acc_lr.append(accuracy_score(yte, pred_lr))
    fold_acc_rf.append(accuracy_score(yte, pred_rf))
    fold_acc_base.append(accuracy_score(yte, pred_base))

y_true  = np.asarray(y_true)
yhat_lr = np.asarray(yhat_lr)
yhat_rf = np.asarray(yhat_rf)

### Evalulation metrics and tests

In [14]:
print("Overall accuracy")
print(f"Logistic regression: {accuracy_score(y_true, yhat_lr)}")
print(f"Random forest: {accuracy_score(y_true, yhat_rf)}")
print(f"Baseline: {accuracy_score(y_true, yhat_base)}")
print()
print("Standard deviation between folds")
print(f"Logistic Reg: {np.std(fold_acc_lr, ddof=1)}")
print(f"Random Forest: {np.std(fold_acc_rf, ddof=1)}")
print(f"Baseline: {np.std(fold_acc_base, ddof=1)}")

Overall accuracy
Logistic regression: 0.7440476190476191
Random forest: 0.7202380952380952
Baseline: 0.625

Standard deviation between folds
Logistic Reg: 0.22039272860726333
Random Forest: 0.26475101764318365
Baseline: 0.19444444444444448


In [15]:
theta, CI, p = mcnemar(y_true, yhat_lr, yhat_rf, alpha=0.05/3)
print("\nMcNemar (LogReg - RF)")
print(f"  θ̂ = {theta:.4f}   95 % CI: {CI}   p-value: {p:.3e}")

Result of McNemars test using alpha= 0.016666666666666666
Comparison matrix n
[[115.  10.]
 [  6.  37.]]
Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] =  (-0.03287211598298223, 0.08040695982177004)
p-value for two-sided test A and B have same accuracy (exact binomial test): p= 0.454498291015625

McNemar (LogReg - RF)
  θ̂ = 0.0238   95 % CI: (-0.03287211598298223, 0.08040695982177004)   p-value: 4.545e-01


In [16]:
theta, CI, p = mcnemar(y_true, yhat_lr, yhat_base, alpha=0.05/3)
print("\nMcNemar (LogReg - Base)")
print(f"  θ = {theta:.4f}   95 % CI: {CI}   p-value: {p:.3e}")

Result of McNemars test using alpha= 0.016666666666666666
Comparison matrix n
[[89. 36.]
 [16. 27.]]
Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] =  (0.018423393188428117, 0.2183395994772046)
p-value for two-sided test A and B have same accuracy (exact binomial test): p= 0.0077874363057435225

McNemar (LogReg - Base)
  θ = 0.1190   95 % CI: (0.018423393188428117, 0.2183395994772046)   p-value: 7.787e-03


In [17]:
theta, CI, p = mcnemar(y_true, yhat_rf, yhat_base, alpha=0.05/3)
print("\nMcNemar (RF - Base)")
print(f"  θ = {theta:.4f}   95 % CI: {CI}   p-value: {p:.3e}")

Result of McNemars test using alpha= 0.016666666666666666
Comparison matrix n
[[88. 33.]
 [17. 30.]]
Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] =  (-0.004083595859486033, 0.193523916065919)
p-value for two-sided test A and B have same accuracy (exact binomial test): p= 0.032839137564268484

McNemar (RF - Base)
  θ = 0.0952   95 % CI: (-0.004083595859486033, 0.193523916065919)   p-value: 3.284e-02
